In [1]:
import os
import numpy as np
import pandas as pd
import addfips
from src.utils.paths import get_parent_dir
from linearmodels.panel import PooledOLS, PanelOLS
import statsmodels.api as sm

In [2]:
pdir = get_parent_dir(2)

### 1) Prepare data

In [3]:
def read_csse(path):
    df = pd.read_csv(path)
    df = df.set_index("Unnamed: 0")
    df.index = pd.to_datetime(df.index)
    return df

def read_sahie(path, granularity='county'):
    df = (pd.read_csv(path, header=68, sep=',')
          .drop(columns=['Unnamed: 25', 'year', 'version',
                          'statefips', 'countyfips', 'geocat'])  
          .apply(lambda s : s.str.strip() if s.dtype is np.object else s)
          .infer_objects()
          )
    # deal with whitespace
    
    # split data: county/state
    if granularity == 'county':
        df = df.query("county_name != ''")
    elif granularity == 'state':
        df = df.query("county_name == ''")
    else: 
        return df
    return df

Read health data

In [4]:
data_dir = os.path.join(pdir, 'data')
path_sahie_raw = os.path.join(data_dir, 'raw', 'health', 'SAHIE_2017.csv')
sahie = read_sahie(path=path_sahie_raw, granularity='all')

outfile = os.path.join(data_dir, 'processed', 'health',
                          'SAHIE_2017_cleaned.csv')
if not os.path.exists(outfile):
    sahie.to_csv(outfile)

/Users/felix/miniconda3/envs/bd4pp/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [5]:
sahie.dtypes

agecat          int64
racecat         int64
sexcat          int64
iprcat          int64
NIPR           object
nipr_moe       object
NUI            object
nui_moe        object
NIC            object
nic_moe        object
PCTUI          object
pctui_moe      object
PCTIC          object
pctic_moe      object
PCTELIG        object
pctelig_moe    object
PCTLIIC        object
pctliic_moe    object
state_name     object
county_name    object
dtype: object

In [6]:
string_cols = ['county_name', 'state_name']
sahie[string_cols] = sahie[string_cols].convert_dtypes()

In [7]:
numeric_cols = ['NIPR', 'nipr_moe', 'NUI',
'nui_moe', 'NIC', 'nic_moe', 'PCTUI', 'pctui_moe', 'PCTIC', 'pctic_moe',
'PCTELIG', 'pctelig_moe', 'PCTLIIC', 'pctliic_moe']

for col in numeric_cols:
    sahie[col] = sahie[col].replace('.', np.nan, regex=True)
    sahie[col] = pd.to_numeric(sahie[col], downcast='signed')
sahie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320298 entries, 0 to 320297
Data columns (total 20 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   agecat       320298 non-null  int64  
 1   racecat      320298 non-null  int64  
 2   sexcat       320298 non-null  int64  
 3   iprcat       320298 non-null  int64  
 4   NIPR         287530 non-null  float64
 5   nipr_moe     287530 non-null  float64
 6   NUI          287530 non-null  float64
 7   nui_moe      287530 non-null  float64
 8   NIC          287530 non-null  float64
 9   nic_moe      287530 non-null  float64
 10  PCTUI        287530 non-null  float64
 11  pctui_moe    287530 non-null  float64
 12  PCTIC        287530 non-null  float64
 13  pctic_moe    287530 non-null  float64
 14  PCTELIG      287530 non-null  float64
 15  pctelig_moe  287530 non-null  float64
 16  PCTLIIC      287530 non-null  float64
 17  pctliic_moe  287530 non-null  float64
 18  state_name   320298 non-

In [8]:
sahie[['county_name', 'state_name']] = sahie[['county_name', 'state_name']].apply(lambda s : s.str.strip())

In [9]:
cols = sahie.columns.to_list()
cols = cols[-2:] + cols[:-2]
sahie = sahie[cols]

In [10]:
# query for county data only
sahie_county_data_only = sahie.query("county_name != ''")
sahie_county_data_only.reset_index(drop=True, inplace=True)

In [11]:
sahie_county_data_only.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301632 entries, 0 to 301631
Data columns (total 20 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   state_name   301632 non-null  string 
 1   county_name  301632 non-null  string 
 2   agecat       301632 non-null  int64  
 3   racecat      301632 non-null  int64  
 4   sexcat       301632 non-null  int64  
 5   iprcat       301632 non-null  int64  
 6   NIPR         271426 non-null  float64
 7   nipr_moe     271426 non-null  float64
 8   NUI          271426 non-null  float64
 9   nui_moe      271426 non-null  float64
 10  NIC          271426 non-null  float64
 11  nic_moe      271426 non-null  float64
 12  PCTUI        271426 non-null  float64
 13  pctui_moe    271426 non-null  float64
 14  PCTIC        271426 non-null  float64
 15  pctic_moe    271426 non-null  float64
 16  PCTELIG      271426 non-null  float64
 17  pctelig_moe  271426 non-null  float64
 18  PCTLIIC      271426 non-

In [12]:
sahie_cleaned = pd.get_dummies(
    sahie_county_data_only,
    columns=['sexcat', 'iprcat', 'agecat', 'racecat'],
    drop_first=True) # only store n-1 dummies to avoid "dummy variable trap"

In [13]:
sahie_cleaned = sahie_cleaned.groupby(['county_name'], as_index=False).first()
sahie_cleaned.rename(columns={'county_name': 'county',
                              'state_name': 'state'},
                     inplace=True)


In [14]:
# add fips
af = addfips.AddFIPS()

sahie_county_fips_codes = []
for i, row in sahie_cleaned.iterrows():
    county_fips_code = af.get_county_fips(county=row.county, state=row.state)
    sahie_county_fips_codes.append(county_fips_code)

sahie_cleaned['FIPS'] = sahie_county_fips_codes

# --> TODO: fix None rows !

In [15]:
#sahie_cleaned[sahie_cleaned['county'] == "Anchorage Borough"]['FIPS'] = '02020'
#print(sahie_cleaned['FIPS'].head(40))

In [16]:
# drop Anchorage Borough with missing FIPS, deal with that later

In [17]:
csse_dir = os.path.join(pdir, 'data', 'processed', 'csse', 'US')

fname_confirmed = "time_series_covid19_confirmed_US_timeseries.csv" 
fname_deaths = "time_series_covid19_deaths_US_timeseries.csv"

path_confirmed = os.path.join(csse_dir, fname_confirmed)
path_deaths = os.path.join(csse_dir, fname_deaths)

In [18]:
ts_confirmed = read_csse(path_confirmed)
ts_deaths = read_csse(path_deaths)

ts_confirmed.index.name = 'time'
ts_deaths.index.name = 'time'

In [19]:
demographic_dir = os.path.join(pdir, 'data', 'raw', 'demography')
popdata = pd.read_csv(os.path.join(demographic_dir, 
                                   "POPEST_2019.csv"),
                      encoding = "ISO-8859-1")

# POPESTIMATE2019: 7/1/2019 resident total population estimate
df_pop = popdata[['STNAME', 'CTYNAME', 'POPESTIMATE2019']]
df_pop_counties = df_pop.query("STNAME != CTYNAME")
df_pop_counties = df_pop_counties.rename(columns={'STNAME': 'state',
                                                  'CTYNAME': 'county',
                                                  'POPESTIMATE2019': 'pop2019_county'})

df_pop_states = df_pop.query("STNAME == CTYNAME")
df_pop_states = df_pop_states.reset_index(drop=True)
df_pop_states = df_pop_states.rename(columns={'STNAME': 'state',
                                              'CTYNAME': 'county',
                                              'POPESTIMATE2019': 'pop2019_state'})
df_pop_states.drop(columns='county', inplace=True)

af = addfips.AddFIPS()

county_fips_codes = []
for i, row in df_pop_counties.iterrows():
    county_fips_code = af.get_county_fips(county=row.county, state=row.state)
    county_fips_codes.append(county_fips_code)
    
state_fips_codes = []
for i, row in df_pop_states.iterrows():
    state_fips_code = af.get_state_fips(state=row.state)
    state_fips_codes.append(state_fips_code)
    
df_pop_counties['FIPS'] = county_fips_codes
df_pop_states['FIPS_state'] = state_fips_codes
print(df_pop_states.head())
print(df_pop_counties.head())

        state  pop2019_state FIPS_state
0     Alabama        4903185         01
1      Alaska         731545         02
2     Arizona        7278717         04
3    Arkansas        3017804         05
4  California       39512223         06
     state          county  pop2019_county   FIPS
1  Alabama  Autauga County           55869  01001
2  Alabama  Baldwin County          223234  01003
3  Alabama  Barbour County           24686  01005
4  Alabama     Bibb County           22394  01007
5  Alabama   Blount County           57826  01009


In [20]:
ts_confirmedT = ts_confirmed.transpose()
ts_confirmedT.index.name = "FIPS"
tsconfm = ts_confirmedT.stack()

In [21]:
tsconfm.name = "confirmed_cases" 
tsconfm = tsconfm.reset_index()

### 2) Merge independent and dependent variables 

In [22]:
df_merged = pd.merge(left=tsconfm.reset_index(),
                     right=df_pop_counties,
                     on='FIPS')
df_merged = pd.merge(left=df_merged,
                     right=df_pop_states,
                     on='state')
df_merged.drop(columns='index', inplace=True)

In [23]:
# merge sahie
df_merged = pd.merge(left=df_merged,
                     right=sahie_cleaned,
                     on='FIPS')

### 3) Construct panel using pandas Multi-index 

In [24]:
# fips => entity FE, time => time FE
panel = df_merged.set_index(['FIPS', 'time'])

In [25]:
# county pop share with respect to state pop
# TODO: meaningful? 
panel['county_pop_share_2019'] = \
    panel['pop2019_county'].divide(panel['pop2019_state'])

In [26]:
panel.head()

confirmed_cases   state_x     county_x  pop2019_county  \
FIPS  time                                                                 
10001 2020-01-22                0  Delaware  Kent County          180786   
      2020-01-23                0  Delaware  Kent County          180786   
      2020-01-24                0  Delaware  Kent County          180786   
      2020-01-25                0  Delaware  Kent County          180786   
      2020-01-26                0  Delaware  Kent County          180786   

                  pop2019_state FIPS_state     county_y   state_y     NIPR  \
FIPS  time                                                                   
10001 2020-01-22         973764         10  Kent County  Delaware  13131.0   
      2020-01-23         973764         10  Kent County  Delaware  13131.0   
      2020-01-24         973764         10  Kent County  Delaware  13131.0   
      2020-01-25         973764         10  Kent County  Delaware  13131.0   
      2020-01-26         973764         10  Kent County  Delaware  13131.0   

                  nipr_moe  ...  iprcat_2  iprcat_3  iprcat_4  iprcat_5  \
FIPS  time                  ...                                           
10001 2020-01-22       0.0  ...         0         0         0         0   
      2020-01-23       0.0  ...         0         0         0         0   
      2020-01-24       0.0  ...         0         0         0         0   
      2020-01-25       0.0  ...         0         0         0         0   
      2020-01-26       0.0  ...         0         0         0         0   

                  agecat_1  agecat_2  agecat_3  agecat_4  agecat_5  \
FIPS  time                                                           
10001 2020-01-22         0         0         0         0         0   
      2020-01-23         0         0         0         0         0   
      2020-01-24         0         0         0         0         0   
      2020-01-25         0         0         0         0         0   
      2020-01-26         0         0         0         0         0   

                  county_pop_share_2019  
FIPS  time                               
10001 2020-01-22               0.185657  
      2020-01-23               0.185657  
      2020-01-24               0.185657  
      2020-01-25               0.185657  
      2020-01-26               0.185657  

[5 rows x 35 columns]

In [27]:
# select vars
panel_subset = panel[
    ['confirmed_cases', 'pop2019_county', 'county_pop_share_2019', 'NIPR', 
     'nipr_moe', 'NUI', 'nui_moe', 'NIC', 'nic_moe', 'PCTUI', 'pctui_moe',
     'PCTIC', 'pctic_moe', 'PCTELIG', 'pctelig_moe', 'PCTLIIC',
     'pctliic_moe', 'sexcat_1', 'sexcat_2',
     'iprcat_1', 'iprcat_2', 'iprcat_3', 'iprcat_4', 'iprcat_5',
     'agecat_1', 'agecat_2', 'agecat_3', 'agecat_4', 'agecat_5']]

### 4) Run pooled and panel regression 
entity fixed effects don't work. this makes sense as the population
shares per county add up to 100% per county.
time fixed effects on the other hand yield the same results as 
the simple pooled regression above, which makes sense because there
is no variation over time in our current data. looking good!

A) Merged with SAHIE data (significantly less data!)

In [39]:
exog_vars = \
    ['pop2019_county', # County population
     #'NIPR', # Number in demographic group for <income category>
     'NUI' # Number uninsured
     #'PCTELIG'
     ] # Percent uninsured in demographic group for all income levels

exog = sm.add_constant(panel_subset[exog_vars])

# pooled regression
mod_pooled = PooledOLS(dependent=panel_subset.confirmed_cases, 
                       exog=exog)
pooled_res = mod_pooled.fit()
print(pooled_res)

# panel regression
mod_panel_entity = PanelOLS(dependent=panel_subset.confirmed_cases, 
                            exog=exog,
                            time_effects=True)
panel_entity_res = mod_panel_entity.fit()
print(panel_entity_res)

/Users/felix/miniconda3/envs/bd4pp/lib/python3.7/site-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PooledOLS Estimation Summary                          
Dep. Variable:        confirmed_cases   R-squared:                        0.0160
Estimator:                  PooledOLS   R-squared (Between):              0.0913
No. Observations:              110200   R-squared (Within):               0.0000
Date:                Tue, Apr 07 2020   R-squared (Overall):              0.0160
Time:                        15:12:20   Log-likelihood                 -8.48e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      897.10
Entities:                        1450   P-value                           0.0000
Avg Obs:                       76.000   Distribution:                F(2,110197)
Min Obs:                       76.000                                           
Max Obs:                       76.000   F-statistic (robust):             897.10
                            

B) Population data only

In [40]:
exog_vars = \
    ['pop2019_county', # County population
     'county_pop_share_2019'] # Percent uninsured in demographic group for all income levels
exog = sm.add_constant(panel_subset[exog_vars])

# pooled regression
mod_pooled = PooledOLS(dependent=panel_subset.confirmed_cases, 
                       exog=exog)
pooled_res = mod_pooled.fit()
print(pooled_res)

# panel regression
mod_panel_entity = PanelOLS(dependent=panel_subset.confirmed_cases, 
                            exog=exog,
                            time_effects=True)
panel_entity_res = mod_panel_entity.fit()
print(panel_entity_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:        confirmed_cases   R-squared:                        0.0148
Estimator:                  PooledOLS   R-squared (Between):              0.0846
No. Observations:              120308   R-squared (Within):               0.0000
Date:                Tue, Apr 07 2020   R-squared (Overall):              0.0148
Time:                        15:12:34   Log-likelihood                -9.208e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      905.56
Entities:                        1583   P-value                           0.0000
Avg Obs:                       76.000   Distribution:                F(2,120305)
Min Obs:                       76.000                                           
Max Obs:                       76.000   F-statistic (robust):             905.56
                            